In [ ]:
def sebremforqubo(Qfull, beta, Niterations, EmbeddingFlag, step):
    # Qfull - QUBO matrix 
    # beta - a temperature parameter 
    # Niterations - the number of iterations to run the algorithm 
    # EmbeddingFlag - a flag to indicate whether to perform an embedding step 
    # Step - a step size

    
    MAX_N_SOLUTIONS = 40000 # the maximum number of solutions to return
    Nqubits = 512 # number of qubits to use
    Nvariables = Qfull.shape[0]
    params = {'num_reads': 10000, 'auto_scale': False} # the number of reads per run
    absJmax = 1
    abshmax = 2
    SampleScaleFactor = 0.99

    RelativeEntropy = np.zeros(Niterations)
    BestObjective = np.zeros(Niterations)

    # Obtain Adjacency matrix and list of qubits used
    
    # affiliate connection flag
    aff = 0
    # Choose a connection depending on the time of day
    import datetime
    now = datetime.datetime.now()
    if now.hour in [4, 9, 14, 19]:
        # Creates a remote SAPI connection handle to the affiliate server
        conn = sapiRemoteConnection('https://206.117.25.108/sapi/','92cc484c78372afc3810749cf106e95c76540571')
        aff = 1
    else:
        # Creates a remote SAPI connection handle to the ISI server
        conn = sapiRemoteConnection('https://206.117.25.107/sapi/','d6d46de897c148729afb655909aff78d0eb125ea')

    # Create a SAPI solver handle
    if aff==1:
        solver = sapiSolver(conn,'V6-LP') # Hardware Low Priority (affiliate)
    else:
        solver = sapiSolver(conn,'V6-MP') # Hardware Medium Priority

    AdjacencyMatrix = getHardwareAdjacency(solver)
    QubitsUsed = GenerateQubitsUsed(AdjacencyMatrix)

    # Shift QubitsUsed to use a different part of the chip
    # QubitsUsed = QubitsUsed - 128; # Shifts everything 2 cells up
    
    # the rest of the algorithm

Start of code

In [ ]:
import numpy as np
from functools import partial

In [ ]:
QubitMapping, couplers, VariableInteraction, h_chimera, J_chimera, hfull, Jfull = InitialEmbedding(Qfull, EmbeddingFlag, AdjacencyMatrix, QubitsUsed)

EmbeddingData = {}
EmbeddingData['Qmap'] = QubitMapping
EmbeddingData['couplers'] = couplers


In [ ]:
# What is Happening here ?

IsingEnergy = lambda S: (S.T@Jfull@S + S.T@hfull)

# OR this one 👇🏻
IsingEnergy = partial(np.dot, np.dot(Jfull, np.transpose(S)) + np.dot(np.transpose(S), hfull))

def IsingEnergy(S):
    return np.dot(np.dot(Jfull, np.transpose(S)) + np.dot(np.transpose(S), hfull))


In [ ]:
for iter in range(Niterations):

    # Solve the Ising model with quantum processor
    answer = IsingConnectSolve(SampleScaleFactor*h_chimera, SampleScaleFactor*J_chimera, params)

    # NumberOfSamples = size(answer.solutions,2);
    NumberOfSamples = answer.solutions.shape[1]

    # Extract a distribution from the samples
    SpinSolutions, ProbabilityOfSamples, EnergiesOfSamples[iter] = ExtractDistribution(answer, QubitMapping)

    # Compute objective function on samples
    Gvec = GvecComputation(SpinSolutions, IsingEnergy)  # Compute the values of the normalized original function on the samples
    G[iter] = Gvec

    # Extract best solutions
    BestObjective[iter], BestSolutions[iter] = FindBestSolution(SpinSolutions, Gvec, Qfull)

    # Compute the relative entropy
    RelativeEntropy[iter] = -ProbabilityOfSamples * np.log(ProbabilityOfSamples) + beta * (Gvec * ProbabilityOfSamples)

    # Compute the gradient of the relative entropy
    GradRE_h, GradRE_J = REGradient(SpinSolutions, ProbabilityOfSamples, Gvec, beta, VariableInteraction)
    Grad[iter, :] = [GradRE_h, GradRE_J]
    if iter > 1:
        GradInnerProduct = Grad[iter, :] * Grad[iter - 1, :] / (np.linalg.norm(Grad[iter - 1, :]) * np.linalg.norm(Grad[iter, :]))
    else:
        GradInnerProduct = 0

    # Update Ising model
    if GradInnerProduct < -0.1:
        Step = max(0.005, Step / 3)
    elif GradInnerProduct > 0.5:
        Step = min(0.1, Step * 3)

    h_chimera, J_chimera = UpdateIsingModel(h_chimera, J_chimera, GradRE_h, GradRE_J, Step, QubitMapping, couplers, BestObjective, BestSolutions, iter)

    h[iter] = h_chimera
    J[iter] = J_chimera
    num_samples[iter] = NumberOfSamples
    step[iter] = Step

    # Start printing information
    DisplayInformation(RelativeEntropy, iter, BestObjective, BestSolutions, beta, Niterations, GradInnerProduct, NumberOfSamples)

    # Save data
    np.savez("SEBREMdata", BestObjective=BestObjective, BestSolutions=BestSolutions, G=G, EnergiesOfSamples=EnergiesOfSamples, Grad=Grad, RelativeEntropy=RelativeEntropy, h=h, J=J)

    EmbeddingData['h'] = h
    EmbeddingData['J'] = J
    EmbeddingData['num_samples'] = num_samples
    EmbeddingData['step'] = step

# Extract best solution
bestindex = np.argmin(BestObjective)
BestObjectiveFound = BestObjective[bestindex]
BestSolutionFound = BestSolutions[bestindex][:, 0]



In [ ]:
import numpy as np
from scipy.optimize import linprog
from scipy.sparse import csgraph

def InitialEmbedding(Qfull, EmbeddingFlag, AdjacencyMatrix, QubitsUsed):
    # Parameters
    Nqubits = 512
    absJmax = 1
    abshmax = 2

    StartingQubit = 217  # First qubit of one of the central cells

    # Convert Qfull to upper triangular
    Qfull = np.triu(Qfull) + np.triu(Qfull.T, 1)

    # Convert to Ising model to allow proper normalization
    hfull, Jfull = quboToIsing(Qfull)  # Now Jfull is also upper triangular

    # Normalize
    NormFactor = max((1 / abshmax) * np.max(np.abs(hfull)), (1 / absJmax) * np.max(np.abs(Jfull)))
    hfull = hfull / NormFactor
    Jfull = Jfull / NormFactor

    # Construct initial embedding
    if EmbeddingFlag == 1:  # Apply the greedy embedding algorithm to find a starting embedding
        QubitMapping, AdjMat = GreedyEmbedding(Jfull, AdjacencyMatrix, StartingQubit)
    elif EmbeddingFlag == 2:  # Apply stochastic version of greedy embedding
        QubitMapping, AdjMat = StochasticGreedyEmbedding(Jfull, AdjacencyMatrix, StartingQubit)
    else:
        QubitMapping, AdjMat = RandomizedDirectEmbedding(Jfull, AdjacencyMatrix, QubitsUsed)

    # Embedded J: AdjMat is only an adjacency matrix
    # J_chimera should be upper triangular (because Jfull is)
    J_chimera = np.zeros((Nqubits, Nqubits))
    h_chimera = np.zeros(Nqubits)

    J_chimera[QubitMapping[:, 1], QubitMapping[:, 1]] = Jfull * AdjMat
    J_chimera = np.triu(J_chimera)  # Keep J_chimera upper triangular

    h_chimera[QubitMapping[:, 1]] = hfull  # Assign local fields

    # Check if there is a zero row/column in J_chimera for which
    # h_chimera is also zero, and add a very small local field if that is the
    # case
    ZeroDetect = np.sum(np.abs(np.column_stack((h_chimera[QubitMapping[:, 1]], np.sum(Jfull * AdjMat + (Jfull * AdjMat).T, axis=1)))), axis=1)
    if np.any(ZeroDetect == 0):
        h_chimera[QubitMapping[ZeroDetect == 0, 1]] = 0.001

    # Inverse QubitMapping
   


In [ ]:
def ExtractDistribution(answer, QubitMapping):
    SpinSolutions = answer["solutions"][QubitMapping[:, 2], :]
    ProbabilityOfSamples = answer["num_occurrences"] / sum(answer["num_occurrences"])
    EnergiesOfSamples = answer["energies"]
    return SpinSolutions, ProbabilityOfSamples, EnergiesOfSamples


In [ ]:
def GreedyEmbedding(J, AdjacencyMatrix, StartingQubit):
    J = J - np.diag(np.diag(J))

    Nvars = J.shape[0]
    Nqubits = 512
    Qubit = StartingQubit

    MissingQubits = np.where((np.sum(AdjacencyMatrix, axis=1)) == 0)[0]
    for i in range(len(MissingQubits)):
        AdjacencyMatrix[MissingQubits[i], :] = 0*AdjacencyMatrix[MissingQubits[i], :]
        AdjacencyMatrix[:, MissingQubits[i]] = 0*AdjacencyMatrix[:, MissingQubits[i]]

    StrongCouplingIndex = np.argmax(np.sum(np.abs(J)))

    VariablesLeft = np.arange(Nvars)
    VariablesAssigned = []

    QubitsAssigned = []
    QubitsLeft = np.arange(Nqubits)
    QubitsLeft = np.delete(QubitsLeft, MissingQubits)

    VariablesAssigned.append(StrongCouplingIndex)
    VariablesLeft = np.delete(VariablesLeft, StrongCouplingIndex)
    J[:, StrongCouplingIndex] = np.zeros(Nvars)

    QubitsAssigned.append(Qubit)

    QubitNeighbors = np.where(AdjacencyMatrix[Qubit, :] != 0)[0]

    for n in range(2, Nvars):
        for i in range(len(VariablesLeft)):
            for j in range(len(QubitNeighbors)):
                Coupling = 0
                for k in range(len(VariablesAssigned)):
                    Coupling += abs(J[VariablesAssigned[k], VariablesLeft[i]])*AdjacencyMatrix[QubitNeighbors[j], QubitsAssigned[k]]
                CouplingStrength[i, j] = Coupling

        MaxCouplingStrength = np.amax(CouplingStrength)
        PossibleNewQubitIndices = np.where(np.amax(CouplingStrength, axis=1) == MaxCouplingStrength)[0]
        for nqi in range(len(PossibleNewQubitIndices)):
            DistanceToLastAssignedQubit[nqi] = abs(QubitsAssigned[-1]-QubitNeighbors[PossibleNewQubitIndices[nqi]])
        MinIndex = np.argmin(DistanceToLastAssignedQubit)
        NewQubit = QubitNeighbors[PossibleNewQubitIndices[MinIndex]]

        NewVarIndex = np.argmax(CouplingStrength[:, NewQubitIndex])
        NewVar = VariablesLeft[NewVarIndex]

        VariablesAssigned.append(NewVar)
        VariablesLeft = np.delete(VariablesLeft, np.where(VariablesLeft == NewVar)[0])

        QubitsAssigned.append(NewQubit)


In [ ]:
def GvecComputation(SpinSolutions, G):
    Gvec = np.apply_along_axis(G, 1, SpinSolutions)
    return Gvec


In [ ]:
def HammingDistance(String1, String2):
distance = sum(np.mod(String1 + String2, 2))
return distance

In [ ]:
def REGradient(SpinSolutions,ProbabilityOfSamples,Gvec,beta,VariableInteraction):
    #RECOMPUTATION COmputes the relative entropy and its gradient
    #   It takes the structure answer as input and computes the relative
    #   entropy between the input QUBO and the embedded one. Note that internally 
    #   the code works in the Ising framework, so solutions are strings of
    #   {+1,-1}. 

    # Notes:
    # -When yo urun the deivce 100 times and you want to know the 

    sampleset["info"]


    
    Nvariables = SpinSolutions.shape[0]

    # Fast gradient computation

    Corr1 = SpinSolutions@ProbabilityOfSamples.T # Vector of mean values of variables

    Corr2 = SpinSolutions@np.diag(ProbabilityOfSamples)@SpinSolutions.T
    VecCorr2 = Corr2.reshape(-1) # Vectorized two-variable correlation matrix

    Corr2coupler = VecCorr2[VariableInteraction[:,0] +(VariableInteraction[:,1] -1)*Nvariables] # Two variable correlations associated with just the couplers


    Hvec = ProbabilityOfSamples*(np.log2(ProbabilityOfSamples) + beta*Gvec)

    ProdVec = SpinSolutions[VariableInteraction[:,0],:]*SpinSolutions[VariableInteraction[:,1],:]

    AverageLog = (np.log2(ProbabilityOfSamples) + beta*Gvec)*ProbabilityOfSamples


    GradRE_h = -beta*(SpinSolutions@Hvec.T - Corr1*AverageLog).T
    GradRE_J = -beta*(ProdVec@Hvec.T - Corr2coupler*AverageLog).T

    return GradRE_h,GradRE_J


In [ ]:
def UpdateIsingModel(h_chimera, J_chimera, GradRE_h, GradRE_J, Step, QubitMapping, couplers, BestObjective, BestSolutions, iter):
    absJmax = 1
    abshmax = 2

    Nvariables = QubitMapping.shape[0]
    MaxCouplers = couplers.shape[0]
    Nqubits = len(h_chimera)

    # First, we need to map the free parameters in h_chimera and J_chimera into
    # a vector of parameters in order to facilitate some of the calculations

    Parameters = np.zeros(Nvariables + MaxCouplers)
    Parameters[:Nvariables] = h_chimera[QubitMapping[:,1]-1]
    Parameters[Nvariables:] = J_chimera[(couplers[:,1]-1)*Nqubits + couplers[:,0]-1]

    GradParameters = np.concatenate((GradRE_h, GradRE_J))

    ParametersMin = np.concatenate((-abshmax*np.ones(Nvariables), -absJmax*np.ones(MaxCouplers)))
    ParametersMax = np.concatenate((abshmax*np.ones(Nvariables), absJmax*np.ones(MaxCouplers)))

    # Find vector of max allowed update steps for all parameters

    I_positive = np.where(GradParameters > 0)
    alpha = np.zeros(Nvariables + MaxCouplers)
    alpha[I_positive] = (Parameters[I_positive] - ParametersMin[I_positive])/GradParameters[I_positive]

    I_negative = np.where(GradParameters < 0)
    alpha[I_negative] = (Parameters[I_negative] - ParametersMax[I_negative])/GradParameters[I_negative]

    I_zero = np.where(GradParameters == 0)
    alpha[I_zero] = 10**10

    # Find the minimum value of allowed update steps

    alpha_min = np.min(alpha)

    # Update the parameters, using an update mask to prevent moving outside the
    # constraint box parameters that are already at the boundary

    UpdatedParameters = Parameters - min(alpha_min,Step)*(GradParameters*(1 - (np.abs(np.abs(Parameters) - ParametersMax) < 10**(-8)) * (np.sign(Parameters*GradParameters) < 0)))

    # Now we need to map back the update parameters into the vector h_chimera
    # and the matrix J_chimera

    h_chimera[QubitMapping[:,1]-1] = UpdatedParameters[:Nvariables]
    J_chimera[(couplers[:,1]-1)*Nqubits + couplers[:,0]-1] = UpdatedParameters[Nvariables:]

    MIXFACTOR = 0.05  # If the new optimum is better, add an Ising model whose ground state corresponds to it.

    if iter > 1 and BestObjective[iter] <= np.min(BestObject


In [ ]:
def ChimeraParametersFromBinarySolution(BinarySolutions, QubitMapping, couplers):
    Nqubits = QubitMapping.shape[0]

    FullSolution = np.zeros(512)
    h = np.zeros(512)
    J = np.zeros((512, 512))

    NumberOfBestSolutions = BinarySolutions.shape[1]

    for soln in range(NumberOfBestSolutions):
        FullSolution[QubitMapping[:,1]] = 2*BinarySolutions[:,soln]-1

        h = h -(0.2/NumberOfBestSolutions)*FullSolution # Check sign

        for i in range(couplers.shape[0]):
            J[couplers[i,0],couplers[i,1]] = J[couplers[i,0],couplers[i,1]] - (1/NumberOfBestSolutions)*FullSolution[couplers[i,0]]*FullSolution[couplers[i,1]]
    return h,J

In [ ]:
def FindBestSolution(SpinSolutions, Gvec, Qfull):
    # Order Gvec
    OrderedGvec = Gvec[np.argsort(Gvec[:,0])]

    # Order SpinSolutions
    sortindices = np.argsort(Gvec[:,0])
    OrderedSpinSolutions = SpinSolutions[:,sortindices]

    # Find how many solutions attain the lowest objective
    NumberOfGroundStates = len(np.where(OrderedGvec[:,0] == OrderedGvec[0,0])[0])

    # Extract best solutions
    BestSolution = (OrderedSpinSolutions[:,0:NumberOfGroundStates] + 1)/2 # Gives a binary string
    BestObjective = np.dot(np.dot(BestSolution[:,0].T, Qfull), BestSolution[:,0])

    return BestObjective, BestSolution




In [ ]:
def DisplayInformation(RelativeEntropy, iter, BestObjective, BestSolutions, beta, Niterations, GradInnerProduct, NumberOfSamples):
    def HammingDistance(str1, str2):
        diffs = 0
        for ch1, ch2 in zip(str1, str2):
            if ch1 != ch2:
                diffs += 1
        return diffs

    if iter == 1:
        print(' ')
        print('SEQUENTIAL EMBEDDING BY RELATIVE ENTROPY MINIMIZATION ')
        print(' ')
        print('Parameters: beta = {}, Number of Iterations = {}'.format(beta, Niterations))
        print(' ')
        print('Iter \t RE \t Best objective \t Hamming Distance \t Gradient angle \t Samples')
        print('{} \t {} \t {}'.format(iter, RelativeEntropy[iter], BestObjective[iter]))
        
    else:
        Hdistance = np.zeros((BestSolutions[iter].shape[1], BestSolutions[iter-1].shape[1]))
        for i in range(BestSolutions[iter].shape[1]):
            for j in range(BestSolutions[iter-1].shape[1]):
                Hdistance[i,j] = HammingDistance(BestSolutions[iter-1][:,j], BestSolutions[iter][:,i])
        hammingdistance = np.min(np.min(Hdistance))
        print('{} \t {} \t {} \t {} \t {} \t {}'.format(iter, RelativeEntropy[iter], BestObjective[iter], hammingdistance, GradInnerProduct, NumberOfSamples))


In [ ]:
def RandomizedDirectEmbedding(J, AdjacencyMatrix, QubitsUsed):
    J = J - np.diag(np.diag(J))

    Nvariables = J.shape[0]
    Nqubits = 512

    MissingQubits = np.where(np.sum(AdjacencyMatrix, axis=0) == 0)[0] # Missing qubits in processor

    #Remove Missing qubits from AdjacencyMatrix (in case they're still there)
    for i in range(len(MissingQubits)):
        AdjacencyMatrix[MissingQubits[i], :] = 0
        AdjacencyMatrix[:, MissingQubits[i]] = 0

    # Create a random mapping form variables to qubits used
    QubitMapping = np.column_stack((np.arange(1,Nvariables+1), np.random.permutation(QubitsUsed)[:Nvariables]))

    # Compute adjacency matrix of mapped variables
    AdjMat = AdjacencyMatrix[QubitMapping[:,1]-1, QubitMapping[:,1]-1]

    return QubitMapping, AdjMat


In [ ]:
import numpy as np

def StochasticGreedyEmbedding(J, AdjacencyMatrix, StartingQubit):
    #GREEDYEMBEDDING Generates an embedding that prioritizes keeping the
    #strongest connections of J.

    # Inputs:
    #
    #  J  : Interaction matrix to be approximated
    #  AdjacencyMatrix : Adjacency matrix of the chip
    #  StartingQubit : Initial qubit to start the embedding (picking one with 
    #                  high connectivity and somwhere around the center of the
    #                  chip recommended)
    #
    #
    # Outputs: 
    #
    #  QuibtMapping : N x 2 matrix, first column is a list of variables of
    #  the quadratic function, second column has the qubits assigned to those
    #  variables
    #  AdjMat : reduced adjacency matrix representing the connections between
    #  the qubits chosen from the embedding

    J = J - np.diag(np.diag(J))

    Nvars = J.shape[0]
    Nqubits = 512  # Total number of qubits in Vesuvius (used for indexing)    
    Qubit = StartingQubit

    MissingQubits = np.where((np.sum(AdjacencyMatrix, axis=1)) == 0)[0] # Missing qubits in processor

    #Remove Missing qubits from AdjacencyMatrix (in case they're still there)
    for i in range(len(MissingQubits)):
        AdjacencyMatrix[MissingQubits[i], :] = 0*AdjacencyMatrix[MissingQubits[i], :]
        AdjacencyMatrix[:, MissingQubits[i]] = 0*AdjacencyMatrix[:, MissingQubits[i]]


    [~, StrongCouplingIndex] = np.max(np.sum(np.abs(J), axis=1)) # Find the index with the strongest coupling

    VariablesLeft = np.arange(1, Nvars+1)
    VariablesAssigned = []

    QubitsAssigned = []
    QubitsLeft = np.arange(1, Nqubits+1)
    QubitsLeft = np.delete(QubitsLeft, MissingQubits)

    VariablesAssigned = [StrongCouplingIndex]
    VariablesLeft = np.delete(VariablesLeft, StrongCouplingIndex-1) # Remove the assigned variable from list of variables
    J[:, StrongCouplingIndex-1] = np.zeros((Nvars, 1))

    QubitsAssigned = [QubitsAssigned, Qubit]

    QubitNeighbors = np.where(AdjacencyMatrix[Qubit-1, :] != 0)[0] # Qubits adjacent to first assigned qubit

    for n in range(2, Nvars+1):

        for i in range(len(VariablesLeft)):
            for j in range(len(QubitNeighbors)):
                Coupling = 0
                for k in range(len(VariablesAssigned)):
                    Coupling = Coupling + np.abs(J[VariablesAssigned[


In [ ]:

def GenerateQubitsUsed(AdjacencyMatrix):
    MissingQubits = np.where(np.sum(AdjacencyMatrix, axis=1) == 0)[0]

    CellsPerSide = 8
    QubitsPerCell = 8

    CellOrder = np.array([[4, 4], [4, 5], [5, 5], [5, 4]])
    ct = 4

    for k in range(int(CellsPerSide/2 -1), 0, -1):
        for j in range(np.min(CellOrder[:,1]), np.max(CellOrder[:,1])+1):
            ct += 1
            CellOrder = np.append(CellOrder, [[k, j]], axis=0)

        for i in range(k+1, CellsPerSide-k+1):
            ct += 1
            CellOrder = np.append(CellOrder, [[i, CellOrder[ct-1,1]]], axis=0)

        for j in range(CellOrder[ct,1]-1, np.min(CellOrder[:,1])-1, -1):
            ct += 1
            CellOrder = np.append(CellOrder, [[CellsPerSide-k+1, j]], axis=0)

        for i in range(CellsPerSide-k, k, -1):
            ct += 1
            CellOrder = np.append(CellOrder, [[i, CellOrder[ct-1,1]]], axis=0)

    HorizontalCell = [5, 1, 6, 2, 7, 3, 8, 4]
    VerticalCell = [1, 5, 2, 6, 3, 7, 4, 8]

    QubitsUsed = ((CellOrder[0,0]-1)*CellsPerSide*8+(CellOrder[0,1]-1)*QubitsPerCell)+VerticalCell

    for k in range(1, CellOrder.shape[0]):

        if CellOrder[k,1] != CellOrder[k,1]:
            QubitsUsed = np.append(QubitsUsed, ((CellOrder[k,0]-1)*CellsPerSide*8+(CellOrder[k,1]-1)*QubitsPerCell)+HorizontalCell)
        else:
            QubitsUsed = np.append(QubitsUsed, ((CellOrder[k,0]-1)*CellsPerSide*8+(CellOrder[k,1]-1)*QubitsPerCell)+VerticalCell)

    for i in range(MissingQubits.shape[0]):
        index = np.where(QubitsUsed == MissingQubits[i])[0]
        QubitsUsed = np.delete(QubitsUsed, index)

    return QubitsUsed



In [1]:

def vec(X):
    m, n = X.shape
    x = X.reshape(m*n, 1)
    return x



ModuleNotFoundError: No module named 'numpy'